In [6]:
import numpy
import scipy.special
import matplotlib.pyplot

In [7]:
# load mnist training data
train_dataPath = '/home/liy/CQU_EIE/dataEngineer/digitReco/data/mnist_train_100.csv'
training_data_file = open(train_dataPath, 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

In [8]:
# load the test data
test_dataPath = '/home/liy/CQU_EIE/dataEngineer/digitReco/data/mnist_test_10.csv'
test_data_file = open(test_dataPath, 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

In [9]:

# neural network class definition
class NeuralNetwork:
    def __init__(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        # set number of nodes in each input, hidden, output layer
        self.i_nodes = input_nodes
        self.h_nodes = hidden_nodes
        self.o_nodes = output_nodes
        # link weight matrices, input_hidden, hidden_output
        self.i_h_weight = numpy.random.normal(0.0, pow(self.h_nodes, -0.5), (self.h_nodes, self.i_nodes))
        self.h_o_weight = numpy.random.normal(0.0, pow(self.o_nodes, -0.5), (self.o_nodes, self.h_nodes))
        # learning rate
        self.lr = learning_rate
        # activation function is the sigmoid function
        self.activation_function = lambda x: scipy.special.expit(x)
        # print('已创建神经网络')
 
    # train the neural network
    def train(self, inputs_list, targets_list):
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        hidden_inputs = numpy.dot(self.i_h_weight, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        final_inputs = numpy.dot(self.h_o_weight, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)
        output_errors = targets-final_outputs
        hidden_errors = numpy.dot(self.h_o_weight.T, output_errors)
        # update the weight
        self.h_o_weight += self.lr*numpy.dot((output_errors*final_outputs*(1.0-final_outputs)), numpy.transpose(hidden_outputs))
        self.i_h_weight += self.lr*numpy.dot((hidden_errors*hidden_outputs*(1.0-hidden_outputs)), numpy.transpose(inputs))
        # print('已完成训练')
 
    # query the neural network
    def query(self, inputs_list):
        inputs = numpy.array(inputs_list, ndmin=2).T
        hidden_inputs = numpy.dot(self.i_h_weight, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        final_inputs = numpy.dot(self.h_o_weight, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)
        # print('已完成测试！')
        return final_outputs

In [10]:
# number of input, hidden, output layer
input_nodes = 784
hidden_nodes = 100
output_nodes = 10
 
# learning rate is 0.3
learning_rate = 0.3
 
# create neural network
n = NeuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

In [11]:
# training the neural network
for record in training_data_list:
    all_values_training = record.split(',')
    inputs = (numpy.asfarray(all_values_training[1:])/255.0*0.99)+0.01
    targets = numpy.zeros(output_nodes)+0.01
    targets[int(all_values_training[0])] = 0.99
    n.train(inputs, targets)


In [12]:
# test the neural network
# load the test data
test_data_file = open("F:\\MNIST_DATA\\MNIST_data_all\\mnist_test.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()
# test
scorecard = []
for record in test_data_list:
    all_values_test = record.split(',')
    correct_label = int(all_values_test[0])
    # print(correct_label, "correct label")
    inputs_test = (numpy.asfarray(all_values_test[1:])/255.0*0.99)+0.01
    output_test = n.query(inputs_test)
    label = numpy.argmax(output_test)
    # print(label, "network's answer")
    if label == correct_label:
        scorecard.append(1)
    else:
        scorecard.append(0)

FileNotFoundError: [Errno 2] No such file or directory: 'F:\\MNIST_DATA\\MNIST_data_all\\mnist_test.csv'

In [ ]:
# calculate the performance score
scorecard_array = numpy.asarray(scorecard)
print('performance = ', scorecard_array.sum()/scorecard_array.size)